In [ ]:
from os import environ
from genai import Client, Credentials

genai_key = environ.get('GENAI_KEY', '')
genai_url = environ.get('GENAI_API', '')
credentials = Credentials(api_key=genai_key, api_endpoint=genai_url)
client = Client(credentials=credentials)

In [ ]:
for model in client.model.list(limit=100).results:
    print(model.model_dump(include=["name", "id"]))

In [ ]:
from genai.schema import (
    TextGenerationParameters,
    TextGenerationReturnOptions,
)

for response in client.text.generation.create(
    model_id="mistralai/mixtral-8x7b-instruct-v0-1",
    inputs=["What is a molecule?", "What is NLP?"],
    parameters=TextGenerationParameters(
        max_new_tokens=150,
        min_new_tokens=20,
        return_options=TextGenerationReturnOptions(input_text=True),
    ),
):
    result = response.results[0]
    print(f"Input Text: {result.input_text}")
    print(f"Generated Text: {result.generated_text}")
    print("")

In [ ]:
# https://ibm.github.io/ibm-generative-ai/v2.3.0/rst_source/examples.extensions.langchain.langchain_chat_stream.html

import pprint
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessageChunk, BaseMessage
from genai.extensions.langchain import LangChainChatInterface
from genai.schema import DecodingMethod, TextGenerationParameters

llm = LangChainChatInterface(
    model_id="mistralai/mistral-7b-instruct-v0-2",
    client=Client(credentials=Credentials.from_env()),
    parameters=TextGenerationParameters(
        decoding_method=DecodingMethod.SAMPLE,
        max_new_tokens=100,
        min_new_tokens=10,
        temperature=0.5,
        top_k=50,
        top_p=1,
    ),
)

prompt = "Describe what is Python in one sentence."
print(f"Request: {prompt}")
final: BaseMessage = BaseMessageChunk(content="", type="") # dummy chunk
first = True
for chunk in llm.stream(
    input=[
        SystemMessage(
            content="""You are a helpful, respectful and honest assistant.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature. If a question does not make
any sense, or is not factually coherent, explain why instead of answering something incorrectly.
If you don't know the answer to a question, please don't share false information.
""",
        ),
        HumanMessage(content=prompt),
    ],
):
    if first:
        first = False
        pprint.pprint(chunk.response_metadata)
    print(chunk.content, end="", flush=True)
    #info = chunk.generation_info
    final = chunk

print("\n\n")
pprint.pprint(final.response_metadata)

In [ ]:
from genai import Client, Credentials
from genai.extensions.langchain import LangChainInterface
from genai.schema import (
    DecodingMethod,
    TextGenerationParameters,
)

llm = LangChainInterface(
    model_id="mistralai/mistral-7b-instruct-v0-2",
    client=Client(credentials=Credentials.from_env()),
    parameters=TextGenerationParameters(
        decoding_method=DecodingMethod.SAMPLE,
        max_new_tokens=500,
        min_new_tokens=10,
        temperature=0.5,
        top_k=50,
        top_p=1,
    ),
)

prompt = "Tell me a knock knock joke."
for chunk in llm.stream(prompt):
    print(chunk, end="", flush=True)